# 3-1_ロジスティック回帰によるガンの悪性腫瘍の分類

## ライブラリのインポート

In [ ]:
# データを処理するための基本的なライブラリ
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
# 訓練データと検証データを分けるライブラリ
from sklearn.model_selection import train_test_split

In [ ]:
# ロジスティック回帰モデルをインポート 
from sklearn.linear_model import LogisticRegression

Google Colaboratory上での出力のデフォルト設定

In [ ]:
# pandasのDataframeの出力
pd.set_option('display.max_columns', 500) # 表示列の最大
pd.set_option('display.max_rows', 500) # 表示行の最大
pd.set_option('display.unicode.east_asian_width', True) # 日本語出力時にヘッダのずれを解消
pd.options.display.float_format = '{:,.5f}'.format # 表示桁数の設定

# ノートブックの表示桁数設定。この設定はprint文には作用せず、セルの最後に書いたものを出力する際に適用されます。
%precision 3
# numpy配列の指数表示禁止設定
np.set_printoptions(suppress=True)
# numpy配列の表示桁数設定
np.set_printoptions(precision=3)

## データの準備
今回使用するデータはscikit-learnからもデータセットとして利用することができる、UCI ML Breast Cancer Wisconsin (Diagnostic) datasetsのコピーです。<br>
UCI Machine Learning Repositoryから公開されています。<br>
downloaded from : https://goo.gl/U2Uwz2

データセットについての説明はこちらに記載されています。<br>
https://scikit-learn.org/stable/datasets/toy_dataset.html#breast-cancer-dataset

#### データを取り込む
- pandasのread_csvメソッドを使用して、mlho/data/cancer.csvファイルを読み込みます
- 読み込んだものは変数df_cancerに代入します

In [ ]:
# csvファイルを読み込みます
df_cancer = pd.read_csv("/content/drive/MyDrive/mlho/data/cancer.csv")

#### データを確認する

In [ ]:
# 読み込んだデータを確認します
# Classが目的変数になる腫瘍の悪性または良性を表しています。（0:悪性、1:良性）
df_cancer.head()

In [ ]:
# df_cancerのデータ要約を確認
df_cancer.info()

In [ ]:
# df_cancerの統計情報を確認
df_cancer.describe()

#### 説明変数、目的変数を切り出す

In [ ]:
# 目的変数にするClass以外をすべて説明変数にする
x = df_cancer.drop(columns='Class')

In [ ]:
x.head(2)

In [ ]:
y = df_cancer['Class']

In [ ]:
y.head(2)

#### データを訓練データと検証データに分割する

**本研修でbreast-cancerデータセットを使用する際には、訓練データ70%、random_state=3で固定しています。他のモデルとの比較をしやすくするためです**

In [ ]:
# 訓練データと検証データに分割(70%を訓練用に使用)
# stratifyは指定した列の値が均等に割り振られる
train_x, val_x, train_y, val_y = train_test_split(x, y, train_size=0.7, test_size=0.3, random_state=3, stratify=y)

## モデルの定義

In [ ]:
model = LogisticRegression(max_iter=2500)

## モデルの学習

In [ ]:
model.fit(train_x, train_y)

## 評価
評価はロジスティック回帰のモデルに用意されているscoreメソッドで行います。

精度の計算方法は分類なのか回帰なのかによって異なりますが、線形回帰の時と同じくscoreメソッドを用います。<br>
scikit-learnに実装されている各モデルのscoreメソッドは、分類の場合は正解率、回帰の場合は決定係数R2で計算されるようになっています。

In [ ]:
# 訓練データで予測精度（正解率）を確認する
model.score(train_x, train_y)

In [ ]:
# 検証データで予測精度（正解率）を確認する
model.score(val_x, val_y)

In [ ]:
# 検証データで予測し、最初の5件の分類確立を確認してみる
model.predict_proba(val_x)[:5]

In [ ]:
# 最初の5件の検証データで予測結果を表示(malignant:悪性(0), benign:良性(1))
model.predict(val_x)[:5]

In [ ]:
# 正解を確認する
val_y.head()

## 予測精度の改善

### データのスケールを揃える
標準化
- データを平均が0、標準偏差(データのばらつき具合)が1になるように調整する
- スケールを揃える為の方法として良く利用される

In [ ]:
# 訓練データ説明変数の各列の平均を計算する
train_x_mean = train_x.mean()
train_x_mean.head()

In [ ]:
# 訓練データ説明変数の各列の標準偏差を計算する
train_x_std = train_x.std()
train_x_std.head()

In [ ]:
# 訓練データの標準化を行う
train_x_scaled = (train_x - train_x_mean) / train_x_std
train_x_scaled.head()

In [ ]:
# 検証データの標準化を行う
val_x_scaled = (val_x - train_x_mean) / train_x_std
val_x_scaled.head()

In [ ]:
# ロジスティック回帰分析の実行
model.fit(train_x_scaled, train_y)

In [ ]:
# 訓練データで予測精度（正解率）を確認する
model.score(train_x_scaled, train_y)

In [ ]:
# 検証データで予測精度（正解率）を確認する
model.score(val_x_scaled, val_y)

## 精度以外の評価指標を確認する
- 適合率（precision）：正と予測したものが、どれだけ正しかったか（高いほど無駄うちが少ない）
- 再現率（recall）： 実際に正であったもののうち、どれだけ正と予測できたか（高いほど取りこぼしが少ない）
- f値（f1-score）：適合率と再現率の調和平均（F値が高い＝適合率も再現率も高い）

In [ ]:
# まとめて評価指標を確認できるclassification_reportをインポート
from sklearn.metrics import classification_report

In [ ]:
# まずは予測を行う
prediction = model.predict(val_x_scaled)

In [ ]:
# classification_reportでまとめて評価指標を確認
# printを使用することでフォーマットが整形されて出力される
print(classification_report(val_y, prediction))

- macro avgはマクロ平均。陽性・陰性を入れ替えて算出した値の平均。
- weighted avg は陽性・陰性を変えて算出した値の加重平均。正解クラスの個数に応じて加重平均する。

（補足）どこから正例とみなすかの閾値はプログラミングで調整をすることができます。predictメソッドは分類確率が0.5より高い方を結果として返しますが、predict_probaメソッドは分類確率自体を返します。例えば、predict_pobaメソッドの結果が0.6以下を0と出力するようにif文でプログラミングすれば、0と判断（悪性の腫瘍と判断される）されやすくして取りこぼしを少なくすることができます。

このノートブックは以上です。